# ライブラリのインポート

In [ ]:
import IPython.display as ipd
from scipy.io import wavfile
import os
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import librosa
import librosa.display 
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
wav_list = ["2023_proto_crack","2023_proto_normal","2023_sa_crack","2023_sa_non_wheez","2023_sa_normal","2023_sa_wheez"]

In [ ]:
# 音声ファイルのパスを指定します。
train_audio_path = '../Sound_Data/'+wav_list[1]+'/'
csv_files = os.listdir(train_audio_path)
csv_files = sorted(csv_files)
print(csv_files)

## Scipyによる音声ファイル読み込み

In [ ]:
# 初期設定値
stetho = "Prototype"#"Sound_Analyzer"
filename = csv_files[4] #wavファイル名
disease = wav_list[1] #受診者の肺音の症状
save_flag = 1 #スペクトログラム画像の保存　0:保存しない　1:保存
axis = 0 #軸とサイズ　0:軸なし、サイズ(1584, 256)　1:軸あり、サイズ比(24, 4)　2:軸なし、サイズデフォルト　3:軸あり、サイズデフォルト
name = filename[0:-4] #+"_part3" #拡張子抜きファイル名

# 音声データの読み込み
# sample_rate:サンプリング周波数, sample:サンプルwavデータ
sample_rate, samples = wavfile.read(train_audio_path + filename)
# Player 実行 モノラルではなくステレオなので配列の向きの違いから転置の必要あり
ipd.Audio(samples.T, rate=sample_rate)

In [ ]:
N = samples.shape[0] # 標本データ数(要素数)を返す
L = len(samples)/sample_rate # 時間(長さ)の算出
padding_list = (sample_rate*20-N)*[0]
plot_time = np.linspace(0, L, N) # グラフ描画のための時間軸の生成


print("file_name:\t"+str(filename))
print("disease:\t"+str(disease))
print("name:\t\t"+str(name))
print("サンプリング周波数:"+str(sample_rate))
print("標本数:\t\t"+str(N))
print("総時間:\t\t"+str(L))
print("データの型:\t"+str(samples.dtype))
print("セグメント:\t"+str(L/132))
print("チャンネル数:\t"+str(samples.ndim))
print("追加の標本数:\t"+str(len(padding_list)))
print("20秒の標本数:\t"+str(sample_rate*20))



# 見附実験用
#print(L/132.0)
#print((L/132.0)*8)
#print(((L/132.0)*8)+(L/132.0))

## WAVファイルの視覚化　：ステレオで両チャンネルを重ね表示

In [ ]:
# wav_plot(ファイル名, 音声データの時間軸, 音声データ, 軸の種類　0:時間軸 1:サンプル軸):
def wav_plot(filename, plot_time, samples, s_num):
    
    # 音声の波形 (sound wave)を視覚化します。
    fig = plt.figure(figsize=(18, 4))
    ax1 = fig.add_subplot(111)
    ax1.set_title('Raw wave of ' + filename)
    ax1.set_ylabel('Amplitude')
    
    if s_num == 0:
        ax1.plot(plot_time, samples)
    else:
        ax1.plot(samples)
        
    #ax1.set_ylim([-10000,10000])
    #plt.savefig("wave_plot/normal/"+filename+".png")

In [ ]:
#横軸　時間　WAVプロット
wav_plot(filename, plot_time, samples, 0)

## ステレオ信号をモノラル信号に変換
2chの平均をとるか・分散の大きい信号を選ぶか

In [ ]:
def ste_to_mono(samples=[], mono='var'):
    
    # 平均
    if mono=='mean':
        mono_samples = np.mean(samples, axis=1) # ステレオを平均してモノラルに変換
        
        
    # 分散の大きい信号
    elif mono=='var' and samples.ndim==2:
        left_samples = samples[:,0].copy()
        right_samples = samples[:,1].copy()
        left_var=left_samples.var()
        right_var=right_samples.var()
        
        if left_var>right_var:
            mono_samples=left_samples
        elif left_var<=right_var:
            mono_samples=right_samples
        else:
            mono_samples=0
    else:
        mono_samples=samples
            
    m_N = samples.shape[0] # 標本数(要素数)を返す
    m_L = N/sample_rate # 時間(長さ)の算出
        
        
    return mono_samples,m_N,m_L

## モノラルに変換した波形の表示

In [ ]:
mono_samples,m_N,m_L = ste_to_mono(samples=samples, mono='var')
#横軸　時間　WAVプロット
wav_plot(filename, plot_time, mono_samples, 0)

## low-passフィルタ

In [ ]:
m_samples = mono_samples
m_samples = np.concatenate([m_samples, np.array(padding_list)])


#m_samples = m_samples[400000:600000]
m_samples.shape

# スペクトログラム変換

In [ ]:
# 短時間フーリエ変換による通常のスペクトログラム
def specgram(audio, sample_rate, window_size=20,step_size=10, eps=1e-10):#スペクトグラム生成
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio, fs=sample_rate,
                                            window='hann',nperseg=nperseg,
                                            noverlap=noverlap,
                                            detrend=False)
    return freqs, times, spec.T.astype(np.float32)

# 短時間フーリエ変換結果に対数を取ったlog-スペクトログラム
def log_specgram(audio, sample_rate, window_size=20, step_size=10, esp=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio, fs=sample_rate,
                                            window='hann',nperseg=nperseg, #窓関数はHANNING窓
                                            noverlap=noverlap,
                                            detrend=False)
    print(esp)
    return freqs, times, np.log(spec.T.astype(np.float32) + esp)

# スペクトログラムの描画
def spec_show(freqs, times, spec, ticks, axis, save, save_flag=0):
    
    #軸の描画
    if axis == 0:#軸なし、サイズ(1584, 256)
        fig = plt.figure(dpi=1,figsize=(1584, 256))
        ax1 = fig.add_subplot(111)
        ax1.axis("off")
        fig.tight_layout()
        fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
        ax1.imshow(spec.T, aspect='auto', origin='lower', extent=[times.min(), times.max(), freqs.min(), freqs.max()])
    elif axis == 1:#軸あり、サイズ(24, 4)
        fig = plt.figure(figsize=(24, 4))
        ax1 = fig.add_subplot(111)
        ax1.imshow(spec.T, aspect='auto', origin='lower', extent=[times.min(), times.max(), freqs.min(), freqs.max()])
        ax1.set_yticks(freqs[::10])
        ax1.set_xticks(times[::50])
        ax1.set_title('spectrogram of ' + filename)
        ax1.set_ylabel('Frequency (Hz)')
        ax1.set_xlabel('Time (s)')
    elif axis == 2:#軸なし、サイズデフォルト
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.axis("off")
        fig.tight_layout()
        fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
        ax1.imshow(spec.T, aspect='auto', origin='lower', extent=[times.min(), times.max(), freqs.min(), freqs.max()])
    elif axis == 3:#軸あり、サイズデフォルト
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.imshow(spec.T, aspect='auto', origin='lower', extent=[times.min(), times.max(), freqs.min(), freqs.max()])
        ax1.set_yticks(freqs[::10])
        ax1.set_xticks(times[::50])
        ax1.set_title('spectrogram of ' + filename)
        ax1.set_ylabel('Frequency (Hz)')
        ax1.set_xlabel('Time (s)')
    else:
        print("not plot image")
        
    # 刻み幅の変更
    if ticks != 0:
        ax1.set_xticks(np.arange(0, times[-1], ticks))

    # 画像の保存
    print("save_flag:"+str(save_flag))
    if save_flag != 0:
        if axis == 1 or axis == 3:#軸あり
            plt.savefig("../Spectrogram/"+stetho+"/axis_"+save+"/axis_"+name+"_"+save+".png")
        elif axis == 0 or axis == 2:#軸なし
            plt.savefig("../Spectrogram/"+stetho+"/"+save+"/"+name+"_"+save+".png")
        else:
            print("not save image")

## 通常のスペクトログラム

In [ ]:
freqs, times, spectrogram = specgram(m_samples, sample_rate)
spec_show(freqs=freqs,times=times,spec=spectrogram, ticks=0, axis=axis, save='spec',save_flag=0)

## logスペクトログラム変換

In [ ]:
freqs, times, log_spectrogram = log_specgram(m_samples, sample_rate)
spec_show(freqs=freqs,times=times,spec=log_spectrogram,ticks=0,axis=axis,save='log_spec', save_flag=save_flag)

In [ ]:
#librosaを利用するために信号データをfloat型に変換
m_samples=m_samples.astype(np.float16)
m_samples

# melスペクトログラム変換

In [ ]:
def mel_spec(m_samples, sample_rate, db, n_mels, n_fft=2048, hop_length=512, win_length=None):
    S = librosa.feature.melspectrogram(y=m_samples, sr=sample_rate, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
    if db=='amp': #振幅をdbスケールに変換
        S_DB = librosa.amplitude_to_db(S, ref=np.max)
    elif db=='power': #振幅の二乗をdbスケールに変換
        S_DB = librosa.power_to_db(S, ref=np.max)
    elif db =='no':
        # 範囲指定してスケーリングする場合
        sc_min = 0
        sc_max = 256
        scaler = MinMaxScaler( (sc_min, sc_max) )
        scaler.fit(S)
        data_scaled = scaler.transform(S)
        S_DB = data_scaled
        print(S_DB)
        print(S_DB.shape)
    else:
        print("can't transform to spectrogram")
        return 0
    return S_DB
    
def mel_spec_show(spec, axis, save, hop_length, y_axis, ticks=0, save_flag=0):
        
    #軸の描画
    if axis == 0:#軸なし、サイズ(1584, 256)
        fig = plt.figure(dpi=1,figsize=(1584, 256))
        ax1 = fig.add_subplot(111)
        ax1.axis("off")
        fig.tight_layout()
        fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
        librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='s', y_axis=y_axis)
    
    elif axis == 1:#軸あり、サイズ(24, 4)
        fig = plt.figure(figsize=(24, 4))
        ax1 = fig.add_subplot(111)
        librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='s', y_axis=y_axis)
        ax1.set_title(filename)
        ax1.set_ylabel('Frequency (Hz)')
        ax1.set_xlabel('Time (s)')
        plt.colorbar(format='%+02.0f dB')
    
    elif axis == 2:#軸なし、サイズデフォルト
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.axis("off")
        fig.tight_layout()
        fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
        librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='s', y_axis=y_axis)
    
    elif axis == 3:#軸あり、サイズデフォルト
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='s', y_axis=y_axis)
        ax1.set_title(filename)
        ax1.set_ylabel('Frequency (Hz)')
        ax1.set_xlabel('Time (s)')
        plt.colorbar(format='%+02.0f dB')
        
    elif axis == 4:#軸あり、サイズ(24, 4)dbなし
        fig = plt.figure(figsize=(24, 4))
        ax1 = fig.add_subplot(111)
        librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='s', y_axis=y_axis)
        ax1.set_title(filename)
        ax1.set_ylabel('Frequency (Hz)')
        ax1.set_xlabel('Time (s)')
    
    else:
        print("not plot image")
        
    # 刻み幅の変更
    if ticks != 0:
        ax1.set_xticks(np.arange(0, times[-1], ticks))

    # 画像の保存
    if save_flag != 0:
        if axis == 1 or axis == 3:
            plt.savefig("../Spectrogram/"+stetho+"/axis_"+save+"/axis_"+name+"_"+save+".png")
        elif axis == 0 or axis == 2:
            plt.savefig("../Spectrogram/"+stetho+"/"+save+"/"+name+"_"+save+".png")
        else:
            print("not save image")
        
        

In [ ]:
#mel_amp_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='amp', n_mels=256)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="mel_amp_spec",hop_length=512,y_axis='log', save_flag=save_flag)

In [ ]:
#mel_amp_breathstep_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='amp', n_mels=256,n_fft=32768)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="mel_amp_breathstep_spec",hop_length=512,y_axis='log', save_flag=save_flag)

In [ ]:
#default_mel_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=128)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="default_mel_spec",hop_length=512,y_axis='mel',save_flag=save_flag)

In [ ]:
#mel_log_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=256, n_fft=1024, hop_length=256, win_length=1024)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="mel_log_spec",hop_length=256,y_axis='log',save_flag=save_flag)

In [ ]:
#mel_log_breathstep_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=256, n_fft=32768, hop_length=256)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="mel_log_breathstep_spec",hop_length=256,y_axis='log', save_flag=save_flag)

In [ ]:
#mel_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=256, n_fft=1024, hop_length=256, win_length=1024)
mel_spec_show(spec=S_DB,ticks=1,axis=axis,save="mel_spec",hop_length=256,y_axis='mel', save_flag=save_flag)

In [ ]:
# default_size_mel_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=256, n_fft=1024, hop_length=256, win_length=1024)
mel_spec_show(spec=S_DB,ticks=1,axis=axis+2,save="default_size_mel_spec",hop_length=256,y_axis='mel', save_flag=save_flag)

#non_db_mel_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='no', n_mels=256, n_fft=1024, hop_length=256, win_length=1024)
mel_spec_show(spec=S_DB,ticks=0,axis=axis,save="non_db_mel_spec",hop_length=256,y_axis='mel', save_flag=save_flag)

#hop1024_mel_log_spec
S_DB = mel_spec(m_samples=m_samples, sample_rate=sample_rate, db='power', n_mels=256, n_fft=1024, hop_length=1024)
mel_spec_show(spec=S_DB,ticks=0,axis=axis,save="mel_log_spec",hop_length=256,y_axis='log',save_flag=save_flag)